# Generate questions

Use the AI model to generate a set of quiz questions.

In [ ]:
import google.generativeai as genai
from google.colab import userdata
import sys

try:
    API_KEY = userdata.get('GOOGLE_API_KEY')
    if not API_KEY:
        raise ValueError("GOOGLE_API_KEY not found in Colab Secrets. Please add it.")
    genai.configure(api_key=API_KEY)
except Exception as e:
    print(f"Error configuring Google Generative AI: {e}", file=sys.stderr)
    print("Please ensure your GOOGLE_API_KEY is set up correctly in Colab Secrets.", file=sys.stderr)
    raise

model = genai.GenerativeModel("gemini-2.0-flash")

def generate_quiz_to_json_format(topic, difficulty, num_questions=15):
    prompt = f"""
Generate {num_questions} {difficulty} multiple-choice questions on the topic "{topic}".
Each question must have exactly 4 options (A, B, C, D), and indicate the correct answer.
For each question, also include its difficulty level.
Provide only the questions in the specified format below, numbered from 1 to {num_questions}, without any introductory or concluding remarks.

Format:
Question 1: ...
Difficulty: {difficulty}
A) ...
B) ...
C) ...
D) ...
Correct Answer: ...

Question 2: ...
Difficulty: {difficulty}
A) ...
B) ...
C) ...
D) ...
Correct Answer: ...

... (up to Question {num_questions})
"""
    response = model.generate_content(prompt)
    return response.text

# Modify the num_questions value here to generate a different number of questions
quiz_raw_text = generate_quiz_to_json_format(topic="Fractions and Geometry", difficulty="easy", num_questions=15)
print(quiz_raw_text)
quiz_raw_text = generate_quiz_to_json_format(topic="Fractions and Geometry", difficulty="medium", num_questions=15)
print(quiz_raw_text)
quiz_raw_text = generate_quiz_to_json_format(topic="Fractions and Geometry", difficulty="hard", num_questions=15)
print(quiz_raw_text)

##Difficulty

In [ ]:
topic = "Fractions and Geometry"
num_questions_per_difficulty = 15 # You can adjust this number

raw_easy_questions = generate_quiz_to_json_format(topic=topic, difficulty="easy", num_questions=num_questions_per_difficulty)
raw_medium_questions = generate_quiz_to_json_format(topic=topic, difficulty="medium", num_questions=num_questions_per_difficulty)
raw_hard_questions = generate_quiz_to_json_format(topic=topic, difficulty="hard", num_questions=num_questions_per_difficulty)

print(raw_easy_questions)
print(raw_medium_questions)
print(raw_hard_questions)

## Format questions

Parse the AI model's response and format the questions into the desired JSON structure (a list of dictionaries, each with 'question', 'options', and 'answer').

In [ ]:
import re

def parse_questions_from_text(raw_text):
    questions = []

    match_start = re.search(r"Question\s*\d*:", raw_text, flags=re.IGNORECASE)
    if match_start:
        raw_text = raw_text[match_start.start():]
    else:
        return []

    question_blocks = re.split(r"Question\s*\d*:\s*", raw_text, flags=re.IGNORECASE)

    for block in question_blocks:
        block = block.strip()
        if not block:
            continue

        lines = block.split("\n")
        if not lines:
            continue

        question_text = ""
        processed_lines = []
        for i, line in enumerate(lines):
            line = line.strip()
            if line:
                if not question_text:
                    question_text = line
                else:
                    processed_lines.append(line)

        if not question_text:
            continue

        options = {}
        correct_answer = ""
        difficulty = ""

        # Parse options, correct answer, and difficulty from the remaining lines
        for line in processed_lines:
            # Use regex to find options, difficulty, and correct answer more flexibly
            option_match = re.match(r"^\s*[A-D]\)\s*(.*)", line, flags=re.IGNORECASE)
            difficulty_match = re.match(r"^\s*\**Difficulty:\s*\**\s*(.*)", line, flags=re.IGNORECASE)
            answer_match = re.match(r"^\s*\**Correct Answer:\s*\**\s*([A-D])", line, flags=re.IGNORECASE)

            if option_match:
                option_key = line.strip()[0].upper()
                options[option_key] = option_match.group(1).strip()
            elif difficulty_match:
                difficulty = difficulty_match.group(1).strip().lower()
            elif answer_match:
                correct_answer = answer_match.group(1).strip().upper()

        # Add question if all essential parts are found (options count might vary if parsing fails)
        if question_text and options and correct_answer and difficulty:
             # Ensure we have all 4 options before adding (optional, depending on desired strictness)
            questions.append({
                 "question": question_text,
                 "options": options,
                 "answer": correct_answer,
                 "difficulty": difficulty
            })
    return questions

# Call the parse_questions_from_text function
formatted_easy_questions = parse_questions_from_text(raw_easy_questions)
formatted_medium_questions = parse_questions_from_text(raw_medium_questions)
formatted_hard_questions = parse_questions_from_text(raw_hard_questions)

print("Formatted Easy Questions:", formatted_easy_questions)
print("Formatted Medium Questions:", formatted_medium_questions)
print("Formatted Hard Questions:", formatted_hard_questions)

## Save to json file

Write the formatted list of questions to a JSON file.

In [ ]:
import json
import os

json_file_path = 'fractions_and_geometry_quiz.json'

if os.path.exists(json_file_path):
    with open(json_file_path, 'r') as f:
        try:
            existing_questions = json.load(f)
        except json.JSONDecodeError:
            print(f"Warning: Could not decode existing JSON from {json_file_path}. Starting with an empty list.", file=sys.stderr)
            existing_questions = []
else:
    existing_questions = []

formatted_easy_questions = parse_questions_from_text(raw_easy_questions)
formatted_medium_questions = parse_questions_from_text(raw_medium_questions)
formatted_hard_questions = parse_questions_from_text(raw_hard_questions)

# Combine all new questions into a single list
all_new_questions = formatted_easy_questions + formatted_medium_questions + formatted_hard_questions

# Append the new questions to the existing list
all_questions = existing_questions + all_new_questions

# Write the combined list back to the JSON file
with open(json_file_path, 'w') as f:
    json.dump(all_questions, f, indent=4)

print(f"Successfully added {len(all_new_questions)} new questions (Easy: {len(formatted_easy_questions)}, Medium: {len(formatted_medium_questions)}, Hard: {len(formatted_hard_questions)}).")
print(f"Total questions in {json_file_path}: {len(all_questions)}")

#AI QUESTIONS

In [19]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import random
import json
import os
import sys

# --- QUIZ UI LOGIC ---
class LiveQuiz:
    def __init__(self, json_file_path='fractions_and_geometry_quiz.json', num_questions=10, difficulty=None):
        self.all_questions = self.load_all_questions_from_json(json_file_path)
        self.questions = self.select_questions(num_questions, difficulty)
        self.current_index = 0
        self.score = 0
        self.answer_input = widgets.Text(description="Your Answer:", placeholder="A/B/C/D")
        self.feedback = widgets.HTML()
        self.next_button = widgets.Button(description="Next")
        self.out = widgets.Output()
        self.answer_input.observe(self.check_answer, names="value")
        self.next_button.on_click(self.next_question)
        if self.questions:
            self.show_question()
        else:
            with self.out:
                print(f"Could not load or select quiz questions from {json_file_path} with the specified criteria. Please ensure the file exists, contains valid data, and matches the difficulty criteria.")
            display(self.out)

    def load_all_questions_from_json(self, json_file_path):
        """Loads all quiz questions from a JSON file."""
        all_questions = []
        if os.path.exists(json_file_path):
            with open(json_file_path, 'r') as f:
                try:
                    all_questions = json.load(f)
                except json.JSONDecodeError:
                    print(f"Error decoding JSON from {json_file_path}", file=sys.stderr)
                    all_questions = []
        else:
            print(f"Error: JSON file not found at {json_file_path}", file=sys.stderr)
        return all_questions

    def select_questions(self, num_questions, difficulty):
        """Selects a specified number of random questions, optionally filtered by difficulty."""
        filtered_questions = self.all_questions

        if difficulty:
            filtered_questions = [q for q in self.all_questions if q.get('difficulty') and q['difficulty'].lower() == difficulty.lower()]
            if not filtered_questions:
                print(f"No questions found for difficulty level: {difficulty}", file=sys.stderr)


        if len(filtered_questions) > num_questions:
            return random.sample(filtered_questions, num_questions)
        else:
            return filtered_questions


    def show_question(self):
        self.answer_input.value = ""
        self.feedback.value = ""
        clear_output(wait=True)
        self.out.clear_output()
        q = self.questions[self.current_index]
        with self.out:
            print(f"Question {self.current_index + 1}: {q['question']}")
            for key, val in q["options"].items():
                print(f"{key}) {val}")
        display(self.out, self.answer_input, self.feedback, self.next_button)

    def check_answer(self, change):
        user_answer = self.answer_input.value.strip().upper()
        correct = self.questions[self.current_index]["answer"]
        if user_answer not in ["A", "B", "C", "D"]:
            self.feedback.value = "<span style='color:red'>Please enter A, B, C, or D</span>"
            return
        if user_answer == correct:
            self.feedback.value = "<span style='color:green'>✅ Correct!</span>"
            self.score += 1
        else:
            self.feedback.value = f"<span style='color:red'>❌ Incorrect. Correct answer: {correct}</span>"

    def next_question(self, btn):
        self.current_index += 1
        if self.current_index < len(self.questions):
            self.show_question()
        else:
            clear_output()
            print(f"🏁 Quiz Completed! Your Score: {self.score}/{len(self.questions)}")

In [ ]:
# --- RUN THE QUIZ ---
quiz = LiveQuiz('fractions_and_geometry_quiz.json', num_questions=5, difficulty='hard')

In [18]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import google.generativeai as genai
import threading
import time
from google.colab import userdata
import sys
import json
import os
import random

try:
    API_KEY = userdata.get('GOOGLE_API_KEY')
    if not API_KEY:
        print("GOOGLE_API_KEY not found in Colab Secrets. AI generation features will not work.", file=sys.stderr)
    else:
        genai.configure(api_key=API_KEY)
except Exception as e:
    print(f"Error configuring Google Generative AI: {e}", file=sys.stderr)
    print("AI generation features will not work.", file=sys.stderr)

class LiveQuizWithTimer:
    def __init__(self, json_file_path='fractions_and_geometry_quiz.json', num_questions=10, time_limit=15, difficulty=None):
        self.all_questions = self.load_all_questions_from_json(json_file_path)
        self.questions = self.select_questions(num_questions, difficulty)
        self.current_index = 0
        self.score = 0
        self.time_limit = time_limit # Set the time limit here
        self.timer_running = False
        self.user_input = widgets.Text(description="Your Answer:")
        self.submit_button = widgets.Button(description="Submit")
        self.output_area = widgets.Output()
        self.timer_label = widgets.Label()
        if self.questions:
            self.display_widgets()
        else:
            with self.output_area:
                print(f"Could not load or select quiz questions from {json_file_path} with the specified criteria. Please ensure the file exists, contains valid data, and matches the difficulty criteria.")
            display(self.output_area)

    def load_all_questions_from_json(self, json_file_path):
        """Loads all quiz questions from a JSON file."""
        all_questions = []
        if os.path.exists(json_file_path):
            with open(json_file_path, 'r') as f:
                try:
                    all_questions = json.load(f)
                except json.JSONDecodeError:
                    print(f"Error decoding JSON from {json_file_path}", file=sys.stderr)
                    all_questions = []
        else:
            print(f"Error: JSON file not found at {json_file_path}", file=sys.stderr)
        return all_questions

    def select_questions(self, num_questions, difficulty):
        """Selects a specified number of random questions, optionally filtered by difficulty."""
        filtered_questions = self.all_questions

        if difficulty:
            filtered_questions = [q for q in self.all_questions if q.get('difficulty') and q['difficulty'].lower() == difficulty.lower()]
            if not filtered_questions:
                print(f"No questions found for difficulty level: {difficulty}", file=sys.stderr)


        if len(filtered_questions) > num_questions:
            return random.sample(filtered_questions, num_questions)
        else:
            return filtered_questions


    def display_widgets(self):
        display(self.output_area, self.timer_label, self.user_input, self.submit_button)
        self.submit_button.on_click(self.check_answer)
        self.show_question()
        self.start_timer()

    def show_question(self):
        with self.output_area:
            clear_output()
            q = self.questions[self.current_index]
            print(f"Q{self.current_index + 1}: {q['question']}")
            for key, val in q["options"].items():
                 print(f"{key}) {val}")
        self.user_input.disabled = False
        self.submit_button.disabled = False


    def start_timer(self):
        self.timer_running = True
        # Ensure only one timer thread is active
        if hasattr(self, 'timer_thread') and self.timer_thread.is_alive():
            self.timer_running = False
            self.timer_thread.join()
        self.timer_thread = threading.Thread(target=self.run_timer)
        self.timer_thread.start()

    def run_timer(self):
        seconds = self.time_limit
        while seconds > 0 and self.timer_running:
            self.timer_label.value = f"⏳ Time Left: {seconds} sec"
            time.sleep(1)
            seconds -= 1
        if self.timer_running:
            self.check_answer(None)

    def check_answer(self, b):
        if self.current_index >= len(self.questions):
            return
        self.timer_running = False

        user_ans = self.user_input.value.strip().upper()
        correct = self.questions[self.current_index]["answer"]
        is_correct = user_ans == correct
        feedback = f"✅ Correct!" if is_correct else f"❌ Wrong! Correct Answer: {correct}"

        if is_correct:
            self.score += 1

        # Disable input and submit button after answer is checked
        self.user_input.disabled = True
        self.submit_button.disabled = True

        with self.output_area:
            clear_output()
            print(feedback)
            time.sleep(2) # Display feedback for a few seconds

        self.current_index += 1 # Move to the next question
        self.user_input.value = "" # Clear previous input


        if self.current_index < len(self.questions):
            self.show_question()
            self.start_timer() # Start timer for the next question
        else:
            with self.output_area:
                clear_output()
                print("🏁 Quiz Completed!")
                print(f"✅ Your Final Score: {self.score}/{len(self.questions)}")
            self.user_input.layout.visibility = 'hidden'
            self.submit_button.layout.visibility = 'hidden'
            self.timer_label.layout.visibility = 'hidden'

In [ ]:
# --- RUN THE QUIZ ---
LiveQuizWithTimer('fractions_and_geometry_quiz.json', time_limit=20)

# Create api endpoints

Define API endpoints using the chosen framework to trigger quiz generation, question retrieval, and answer checking.

In [20]:
from fastapi import FastAPI, HTTPException, Depends
from pydantic import BaseModel
import json
import os
import random
import sys

JSON_FILE_PATH = 'fractions_and_geometry_quiz.json'

class Question(BaseModel):
    question: str
    options: dict[str, str]
    answer: str
    difficulty: str

class UserAnswer(BaseModel):
    question_index: int
    user_option: str

app = FastAPI()

def load_all_questions_from_json(json_file_path: str = JSON_FILE_PATH):
    """Loads all quiz questions from a JSON file."""
    all_questions = []
    if os.path.exists(json_file_path):
        with open(json_file_path, 'r') as f:
            try:
                all_questions = json.load(f)
            except json.JSONDecodeError:
                print(f"Error decoding JSON from {json_file_path}", file=sys.stderr)
                all_questions = []
    else:
        print(f"Error: JSON file not found at {json_file_path}", file=sys.stderr)
    return all_questions

def select_questions(all_questions: list, num_questions: int = 10, difficulty: str = None):
    """Selects a specified number of random questions, optionally filtered by difficulty."""
    filtered_questions = all_questions

    if difficulty:
        # Filter questions by the specified difficulty
        filtered_questions = [q for q in all_questions if q.get('difficulty') and q['difficulty'].lower() == difficulty.lower()]
        if not filtered_questions:
            print(f"No questions found for difficulty level: {difficulty}", file=sys.stderr)

    # Randomly select num_questions from the filtered questions
    if len(filtered_questions) > num_questions:
        return random.sample(filtered_questions, num_questions)
    else:
        return filtered_questions

# Load all questions when the application starts
all_quiz_questions = load_all_questions_from_json()

@app.get("/questions/", response_model=list[Question])
def get_questions(num_questions: int = 10, difficulty: str = None):
    """Retrieve a list of quiz questions."""
    selected_questions = select_questions(all_quiz_questions, num_questions, difficulty)
    if not selected_questions:
        raise HTTPException(status_code=404, detail="No questions found for the specified criteria.")
    return selected_questions

@app.post("/answer/")
def check_user_answer(answer: UserAnswer):
    """Check the user's answer against the correct answer."""
    if answer.question_index < 0 or answer.question_index >= len(all_quiz_questions):
        raise HTTPException(status_code=400, detail="Invalid question index.")

    correct_answer = all_quiz_questions[answer.question_index]["answer"]
    is_correct = answer.user_option.strip().upper() == correct_answer.strip().upper()

    return {"question_index": answer.question_index, "user_answer": answer.user_option, "is_correct": is_correct, "correct_answer": correct_answer}

## Integrate quiz logic

Connect the API endpoints to the refactored quiz code.

In [ ]:
import os
from google.colab import userdata

try:
    api_key = userdata.get('GOOGLE_API_KEY')
    if api_key:
        os.environ['GOOGLE_API_KEY'] = api_key
        print("GOOGLE_API_KEY environment variable set from Colab Secrets.")
    else:
        print("GOOGLE_API_KEY not found in Colab Secrets. Please add it.", file=sys.stderr)
except Exception as e:
    print(f"Error accessing Colab Secrets: {e}", file=sys.stderr)

In [ ]:
from fastapi import FastAPI, HTTPException, Depends
from pydantic import BaseModel
import json
import os
import random
import sys
import re

JSON_FILE_PATH = 'fractions_and_geometry_quiz.json'

class Question(BaseModel):
    question: str
    options: dict[str, str]
    answer: str
    difficulty: str

class UserAnswer(BaseModel):
    question_index: int
    user_option: str

app = FastAPI()

import google.generativeai as genai
try:
    API_KEY = os.getenv('GOOGLE_API_KEY')
    if API_KEY:
        genai.configure(api_key=API_KEY)
    else:
         print("Warning: GOOGLE_API_KEY not found in environment variables. AI generation will not work.", file=sys.stderr)
except Exception as e:
     print(f"Error configuring Google Generative AI: {e}", file=sys.stderr)
     print("AI generation features will not work.", file=sys.stderr)

try:
    model = genai.GenerativeModel("gemini-2.0-flash")
except NameError:
    print("Warning: genai model not initialized. AI generation will not work.", file=sys.stderr)
    model = None

def generate_quiz_to_json_format(topic: str, difficulty: str, num_questions: int = 15):
    """Generates raw quiz text using the AI model."""

    print(f"Generating quiz for topic: {topic}, difficulty: {difficulty}, count: {num_questions}")
    return ""

def parse_questions_from_text(raw_text: str):
    """Parses raw text output from AI into a list of question dictionaries."""
    questions = []

    match_start = re.search(r"Question\s*\d*:", raw_text, flags=re.IGNORECASE)
    if match_start:
        raw_text = raw_text[match_start.start():]
    else:
        return []

    question_blocks = re.split(r"Question\s*\d*:\s*", raw_text, flags=re.IGNORECASE)

    for block in question_blocks:
        block = block.strip()
        if not block:
            continue

        lines = block.split("\n")
        if not lines:
            continue

        question_text = ""
        processed_lines = []
        for i, line in enumerate(lines):
            line = line.strip()
            if line:
                if not question_text:
                    question_text = line
                else:
                    processed_lines.append(line)

        if not question_text:
            continue

        options = {}
        correct_answer = ""
        difficulty = ""

        for line in processed_lines:
            option_match = re.match(r"^\s*[A-D]\)\s*(.*)", line, flags=re.IGNORECASE)
            difficulty_match = re.match(r"^\s*\**Difficulty:\s*\**\s*(.*)", line, flags=re.IGNORECASE)
            answer_match = re.match(r"^\s*\**Correct Answer:\s*\**\s*([A-D])", line, flags=re.IGNORECASE)

            if option_match:
                option_key = line.strip()[0].upper()
                options[option_key] = option_match.group(1).strip()
            elif difficulty_match:
                difficulty = difficulty_match.group(1).strip().lower()
            elif answer_match:
                correct_answer = answer_match.group(1).strip().upper()

        if question_text and options and correct_answer and difficulty:
            questions.append({
                 "question": question_text,
                 "options": options,
                 "answer": correct_answer,
                 "difficulty": difficulty
            })
    return questions


def load_all_questions_from_json(json_file_path: str = JSON_FILE_PATH):
    """Loads all quiz questions from a JSON file."""
    all_questions = []
    if os.path.exists(json_file_path):
        with open(json_file_path, 'r') as f:
            try:
                all_questions = json.load(f)
            except json.JSONDecodeError:
                print(f"Error decoding JSON from {json_file_path}", file=sys.stderr)
                all_questions = []
        print(f"Loaded {len(all_questions)} questions from {json_file_path}")
    else:
        print(f"Warning: JSON file not found at {json_file_path}. Starting with no pre-loaded questions.", file=sys.stderr)
    return all_questions

def select_questions(all_questions: list, num_questions: int = 10, difficulty: str = None):
    """Selects a specified number of random questions, optionally filtered by difficulty."""
    filtered_questions = all_questions

    if difficulty:
        filtered_questions = [q for q in all_questions if q.get('difficulty') and q['difficulty'].lower() == difficulty.lower()]
        if not filtered_questions:
            print(f"No questions found for difficulty level: {difficulty}", file=sys.stderr)

    if len(filtered_questions) > num_questions:
        return random.sample(filtered_questions, num_questions)
    else:
        return filtered_questions

all_quiz_questions = load_all_questions_from_json()


@app.get("/questions/", response_model=list[Question])
def get_questions(num_questions: int = 10, difficulty: str = None):
    """Retrieve a list of quiz questions based on difficulty and number requested."""
    selected_questions = select_questions(all_quiz_questions, num_questions, difficulty)

    if not selected_questions:
         raise HTTPException(status_code=404, detail=f"No questions found for the specified criteria (difficulty: {difficulty}, count: {num_questions}). Consider generating more questions and saving them to {JSON_FILE_PATH}.")
    return [Question(**q) for q in selected_questions]

@app.post("/answer/")
def check_user_answer(answer: UserAnswer):
    """Check the user's answer against the correct answer."""
    if answer.question_index < 0 or answer.question_index >= len(all_quiz_questions):
        raise HTTPException(status_code=400, detail="Invalid question index.")

    correct_question_data = all_quiz_questions[answer.question_index]
    correct_answer = correct_question_data.get("answer", "").strip().upper()

    if not correct_answer:
        raise HTTPException(status_code=500, detail=f"Could not retrieve correct answer for question index {answer.question_index}.")


    is_correct = answer.user_option.strip().upper() == correct_answer
    return {"question_index": answer.question_index, "user_answer": answer.user_option, "is_correct": is_correct, "correct_answer": correct_answer}

## Add dependencies

Create a `requirements.txt` file listing all necessary libraries for the FastAPI application and the quiz logic.

In [ ]:
%%writefile requirements.txt
fastapi
pydantic
uvicorn
python-dotenv
google-generativeai
requests

## Add a production-ready wsgi server

Include a library like Gunicorn in `requirements.txt` and create a simple entry point file (e.g., `wsgi.py`) for the WSGI server.


In [ ]:
%%writefile requirements.txt
fastapi
pydantic
uvicorn
python-dotenv
google-generativeai
requests
gunicorn

In [ ]:
%%writefile wsgi.py
from main import app

## Update the api code

Modify the Flask app to be compatible with a WSGI server and consider how to handle environment variables for configuration (like `SECRET_KEY`).


In [ ]:
%pip install python-dotenv

In [ ]:
from fastapi import FastAPI, HTTPException, Depends
from pydantic import BaseModel
import json
import os
import random
import sys
import re
from dotenv import load_dotenv

load_dotenv()

JSON_FILE_PATH = os.getenv('JSON_FILE_PATH', 'fractions_and_geometry_quiz.json')

GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

# Configure the Gemini API if the key is available
import google.generativeai as genai
if GOOGLE_API_KEY:
    try:
        genai.configure(api_key=GOOGLE_API_KEY)
        print("Google Generative AI configured successfully.")
    except Exception as e:
        print(f"Error configuring Google Generative AI: {e}", file=sys.stderr)
        print("AI generation features will not work.", file=sys.stderr)
    try:
         model = genai.GenerativeModel("gemini-2.0-flash")
         print("Gemini model initialized.")
    except NameError:
         print("Warning: genai model not initialized. AI generation will not work.", file=sys.stderr)
         model = None
else:
    print("Warning: GOOGLE_API_KEY not found in environment variables. AI generation features will not work.", file=sys.stderr)
    model = None


class Question(BaseModel):
    question: str
    options: dict[str, str]
    answer: str
    difficulty: str

class UserAnswer(BaseModel):
    question_index: int
    user_option: str

app = FastAPI()

def generate_quiz_to_json_format(topic: str, difficulty: str, num_questions: int = 15):
    """Generates raw quiz text using the AI model."""
    if model is None:
        print("AI model not available. Cannot generate quiz.", file=sys.stderr)
        return ""
    prompt = f"""
Generate {num_questions} {difficulty} multiple-choice questions on the topic "{topic}".
Each question must have exactly 4 options (A, B, C, D), and indicate the correct answer.
For each question, also include its difficulty level.
Provide only the questions in the specified format below, numbered from 1 to {num_questions}, without any introductory or concluding remarks.

Format:
Question 1: ...
Difficulty: {difficulty}
A) ...
B) ...
C) ...
D) ...
Correct Answer: ...

Question 2: ...
Difficulty: {difficulty}
A) ...
B) ...
C) ...
D) ...
Correct Answer: ...

... (up to Question {num_questions})
"""
    try:
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        print(f"Error during AI content generation: {e}", file=sys.stderr)
        return ""


def parse_questions_from_text(raw_text: str):
    """Parses raw text output from AI into a list of question dictionaries."""
    questions = []

    match_start = re.search(r"Question\s*\d*:", raw_text, flags=re.IGNORECASE)
    if match_start:
        raw_text = raw_text[match_start.start():]
    else:
        return []

    question_blocks = re.split(r"Question\s*\d*:\s*", raw_text, flags=re.IGNORECASE)

    for block in question_blocks:
        block = block.strip()
        if not block:
            continue

        lines = block.split("\n")
        if not lines:
            continue

        question_text = ""
        processed_lines = []
        for i, line in enumerate(lines):
            line = line.strip()
            if line:
                if not question_text:
                    question_text = line
                else:
                    processed_lines.append(line)

        if not question_text:
            continue

        options = {}
        correct_answer = ""
        difficulty = ""

        for line in processed_lines:
            option_match = re.match(r"^\s*[A-D]\)\s*(.*)", line, flags=re.IGNORECASE)
            difficulty_match = re.match(r"^\s*\**Difficulty:\s*\**\s*(.*)", line, flags=re.IGNORECASE)
            answer_match = re.match(r"^\s*\**Correct Answer:\s*\**\s*([A-D])", line, flags=re.IGNORECASE)

            if option_match:
                option_key = line.strip()[0].upper()
                options[option_key] = option_match.group(1).strip()
            elif difficulty_match:
                difficulty = difficulty_match.group(1).strip().lower()
            elif answer_match:
                correct_answer = answer_match.group(1).strip().upper()

        if question_text and options and correct_answer and difficulty:
            questions.append({
                 "question": question_text,
                 "options": options,
                 "answer": correct_answer,
                 "difficulty": difficulty
            })
    return questions


# Helper function to load questions from the JSON file
def load_all_questions_from_json(json_file_path: str = JSON_FILE_PATH):
    """Loads all quiz questions from a JSON file."""
    all_questions = []
    if os.path.exists(json_file_path):
        with open(json_file_path, 'r') as f:
            try:
                all_questions = json.load(f)
            except json.JSONDecodeError:
                print(f"Error decoding JSON from {json_file_path}", file=sys.stderr)
                all_questions = []
        print(f"Loaded {len(all_questions)} questions from {json_file_path}")
    else:
        print(f"Warning: JSON file not found at {json_file_path}. Starting with no pre-loaded questions.", file=sys.stderr)
    return all_questions

# Helper function to select questions based on difficulty and count
def select_questions(all_questions: list, num_questions: int = 10, difficulty: str = None):
    """Selects a specified number of random questions, optionally filtered by difficulty."""
    filtered_questions = all_questions

    if difficulty:
        filtered_questions = [q for q in all_questions if q.get('difficulty') and q['difficulty'].lower() == difficulty.lower()]
        if not filtered_questions:
            print(f"No questions found for difficulty level: {difficulty}", file=sys.stderr)

    if len(filtered_questions) > num_questions:
        return random.sample(filtered_questions, num_questions)
    else:
        return filtered_questions

all_quiz_questions = load_all_questions_from_json()


@app.get("/questions/", response_model=list[Question])
def get_questions(num_questions: int = 10, difficulty: str = None):
    """Retrieve a list of quiz questions based on difficulty and number requested."""
    selected_questions = select_questions(all_quiz_questions, num_questions, difficulty)

    if not selected_questions:
         raise HTTPException(status_code=404, detail=f"No questions found for the specified criteria (difficulty: {difficulty}, count: {num_questions}). Consider generating more questions and saving them to {JSON_FILE_PATH}.")

    # Convert the selected question dictionaries to Pydantic models
    return [Question(**q) for q in selected_questions]

@app.post("/answer/")
def check_user_answer(answer: UserAnswer):
    """Check the user's answer against the correct answer."""

    if answer.question_index < 0 or answer.question_index >= len(all_quiz_questions):
        raise HTTPException(status_code=400, detail="Invalid question index.")

    correct_question_data = all_quiz_questions[answer.question_index]
    correct_answer = correct_question_data.get("answer", "").strip().upper()

    if not correct_answer:
         raise HTTPException(status_code=500, detail=f"Could not retrieve correct answer for question index {answer.question_index}.")


    is_correct = answer.user_option.strip().upper() == correct_answer

    return {"question_index": answer.question_index, "user_answer": answer.user_option, "is_correct": is_correct, "correct_answer": correct_answer}

## Containerize the application (optional but recommended)

Create a `Dockerfile` to define the application's environment and dependencies for easier deployment.


In [ ]:
%%writefile Dockerfile
FROM python:3.9-slim

WORKDIR /app

COPY requirements.txt .

# Install any needed dependencies specified in requirements.txt
RUN pip install --no-cache-dir -r requirements.txt

# Copy the rest of the application code into the working directory
COPY . .

# Make port 80 available to the world outside this container
EXPOSE 80

# Run the application using Gunicorn and Uvicorn workers
# Use the wsgi.py entry point and bind to port 80
CMD ["gunicorn", "-w", "4", "-k", "uvicorn.workers.UvicornWorker", "wsgi:app", "-b", "0.0.0.0:80"]

## Implement logging and monitoring

In [29]:
@app.get("/questions/", response_model=list[Question])
def get_questions(num_questions: int = 10, difficulty: str = None):
    """Retrieve a list of quiz questions based on difficulty and number requested."""
    logging.info(f"Parameters received for /questions/: num_questions={num_questions}, difficulty={difficulty}")

    selected_questions = select_questions(all_quiz_questions, num_questions, difficulty)

    if not selected_questions:
         logging.warning(f"GET /questions/ failed: No questions found for criteria (difficulty: {difficulty}, count: {num_questions}).")
         raise HTTPException(status_code=404, detail=f"No questions found for the specified criteria (difficulty: {difficulty}, count: {num_questions}). Consider generating more questions and saving them to {JSON_FILE_PATH}.")

    logging.info(f"Successfully retrieved {len(selected_questions)} questions for /questions/.")
    return [Question(**q) for q in selected_questions]

@app.post("/answer/")
def check_user_answer(answer: UserAnswer):
    """Check the user's answer against the correct answer."""
    logging.info(f"Parameters received for /answer/: question_index={answer.question_index}, user_option={answer.user_option}")

    if answer.question_index < 0 or answer.question_index >= len(all_quiz_questions):
        logging.warning(f"POST /answer/ failed: Invalid question index received: {answer.question_index}")
        raise HTTPException(status_code=400, detail="Invalid question index.")

    correct_question_data = all_quiz_questions[answer.question_index]
    correct_answer = correct_question_data.get("answer", "").strip().upper()

    if not correct_answer:
         logging.error(f"POST /answer/ failed: Could not retrieve correct answer for question index {answer.question_index}.")
         raise HTTPException(status_code=500, detail=f"Could not retrieve correct answer for question index {answer.question_index}.")


    is_correct = answer.user_option.strip().upper() == correct_answer
    logging.info(f"POST /answer/ result for index {answer.question_index}: is_correct={is_correct}")

    return {"question_index": answer.question_index, "user_answer": answer.user_option, "is_correct": is_correct, "correct_answer": correct_answer}

## Secure the api

Implement security best practices, such as input validation and potentially authentication/authorization if needed.

In [30]:
from fastapi import FastAPI, HTTPException, Depends, Query
from pydantic import BaseModel, Field, validator, field_validator
import json
import os
import random
import sys
import re
from dotenv import load_dotenv
import logging

logging.basicConfig(level=logging.INFO, stream=sys.stdout, format='%(asctime)s - %(levelname)s - %(message)s')

load_dotenv()

JSON_FILE_PATH = os.getenv('JSON_FILE_PATH', 'fractions_and_geometry_quiz.json')

GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

import google.generativeai as genai
if GOOGLE_API_KEY:
    try:
        genai.configure(api_key=GOOGLE_API_KEY)
        logging.info("Google Generative AI configured successfully.")
    except Exception as e:
        logging.error(f"Error configuring Google Generative AI: {e}", exc_info=True)
        print("AI generation features will not work.", file=sys.stderr)
    try:
         model = genai.GenerativeModel("gemini-2.0-flash")
         logging.info("Gemini model initialized.")
    except NameError:
         logging.warning("genai model not initialized. AI generation will not work.")
         model = None
    except Exception as e:
         logging.error(f"Error initializing Gemini model: {e}", exc_info=True)
         model = None
else:
    logging.warning("GOOGLE_API_KEY not found in environment variables. AI generation features will not work.")
    model = None


# Pydantic model for a single quiz question
class Question(BaseModel):
    question: str = Field(..., min_length=1)
    options: dict[str, str]
    answer: str = Field(..., min_length=1, max_length=1)
    difficulty: str = Field(..., min_length=1)

    @field_validator('options')
    def validate_options(cls, v):
        if not v or len(v) != 4:
            raise ValueError('Must provide exactly 4 options')
        valid_keys = {'A', 'B', 'C', 'D'}
        if set(v.keys()) != valid_keys:
             raise ValueError('Option keys must be A, B, C, and D')
        for key, value in v.items():
            if not isinstance(value, str) or not value.strip():
                 raise ValueError(f'Option {key} must be a non-empty string')
        return v

    @field_validator('answer')
    def validate_answer(cls, v, info):
        if 'options' in info.data and v not in info.data['options']:
            raise ValueError('Correct answer must be one of the provided options (A, B, C, or D)')
        return v


# Pydantic model for a user's answer with validation
class UserAnswer(BaseModel):
    question_index: int = Field(..., ge=0)
    user_option: str = Field(..., min_length=1, max_length=1)

    @field_validator('user_option')
    def validate_user_option(cls, v):
        if v.upper() not in {'A', 'B', 'C', 'D'}:
            raise ValueError('User option must be A, B, C, or D')
        return v.upper()


app = FastAPI()

def generate_quiz_to_json_format(topic: str, difficulty: str, num_questions: int = 15):
    """Generates raw quiz text using the AI model."""
    if model is None:
        logging.warning("AI model not available. Cannot generate quiz.")
        return ""

    prompt = f"""
Generate {num_questions} {difficulty} multiple-choice questions on the topic "{topic}".
Each question must have exactly 4 options (A, B, C, D), and indicate the correct answer.
For each question, also include its difficulty level.
Provide only the questions in the specified format below, numbered from 1 to {num_questions}, without any introductory or concluding remarks.

Format:
Question 1: ...
Difficulty: {difficulty}
A) ...
B) ...
C) ...
D) ...
Correct Answer: ...

Question 2: ...
Difficulty: {difficulty}
A) ...
B) ...
C) ...
D) ...
Correct Answer: ...

... (up to Question {num_questions})
"""
    try:
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        logging.error(f"Error during AI content generation: {e}", exc_info=True)
        return ""


def parse_questions_from_text(raw_text: str):
    """Parses raw text output from AI into a list of question dictionaries."""
    questions = []

    match_start = re.search(r"Question\s*\d*:", raw_text, flags=re.IGNORECASE)
    if match_start:
        raw_text = raw_text[match_start.start():]
    else:
        return []

    question_blocks = re.split(r"Question\s*\d*:\s*", raw_text, flags=re.IGNORECASE)

    for block in question_blocks:
        block = block.strip()
        if not block:
            continue

        lines = block.split("\n")
        if not lines:
            continue

        question_text = ""
        processed_lines = []
        for i, line in enumerate(lines):
            line = line.strip()
            if line:
                if not question_text:
                    question_text = line
                else:
                    processed_lines.append(line)

        if not question_text:
            continue

        options = {}
        correct_answer = ""
        difficulty = ""

        for line in processed_lines:
            option_match = re.match(r"^\s*[A-D]\)\s*(.*)", line, flags=re.IGNORECASE)
            difficulty_match = re.match(r"^\s*\**Difficulty:\s*\**\s*(.*)", line, flags=re.IGNORECASE)
            answer_match = re.match(r"^\s*\**Correct Answer:\s*\**\s*([A-D])", line, flags=re.IGNORECASE)

            if option_match:
                option_key = line.strip()[0].upper()
                options[option_key] = option_match.group(1).strip()
            elif difficulty_match:
                difficulty = difficulty_match.group(1).strip().lower()
            elif answer_match:
                correct_answer = answer_match.group(1).strip().upper()

        try:
            question_data = {
                 "question": question_text,
                 "options": options,
                 "answer": correct_answer,
                 "difficulty": difficulty
            }
            validated_question = Question(**question_data)
            questions.append(validated_question.model_dump())
        except Exception as e:
            logging.warning(f"Skipping question due to parsing/validation error: {e} - Raw block: {block[:100]}...", exc_info=True)
            continue


    return questions


def load_all_questions_from_json(json_file_path: str = JSON_FILE_PATH):
    """Loads all quiz questions from a JSON file."""
    all_questions = []
    if os.path.exists(json_file_path):
        with open(json_file_path, 'r') as f:
            try:
                raw_questions = json.load(f)
                for q_data in raw_questions:
                    try:
                        validated_question = Question(**q_data)
                        all_questions.append(validated_question.model_dump())
                    except Exception as e:
                         logging.warning(f"Skipping question from JSON due to validation error: {e} - Data: {q_data.get('question', 'N/A')[:50]}...", exc_info=True)
                         continue
            except json.JSONDecodeError:
                logging.error(f"Error decoding JSON from {json_file_path}", exc_info=True)
                all_questions = []
        logging.info(f"Loaded {len(all_questions)} valid questions from {json_file_path}")
    else:
        logging.warning(f"JSON file not found at {JSON_FILE_PATH}. Starting with no pre-loaded questions.")
    return all_questions

def select_questions(all_questions: list, num_questions: int = 10, difficulty: str = None):
    """Selects a specified number of random questions, optionally filtered by difficulty."""
    filtered_questions = all_questions

    if difficulty:
        filtered_questions = [q for q in all_questions if q.get('difficulty') and q['difficulty'].lower() == difficulty.lower()]
        if not filtered_questions:
            logging.warning(f"No questions found for difficulty level: {difficulty}")

    if len(filtered_questions) > num_questions:
        return random.sample(filtered_questions, num_questions)
    else:
        return filtered_questions

all_quiz_questions = load_all_questions_from_json()


@app.get("/questions/", response_model=list[Question])
def get_questions(num_questions: int = Query(10, ge=1, le=50),
                  difficulty: str = Query(None, min_length=1)
                 ):
    """Retrieve a list of quiz questions based on difficulty and number requested."""
    logging.info(f"GET /questions/ request received with num_questions={num_questions}, difficulty={difficulty}")

    selected_questions = select_questions(all_quiz_questions, num_questions, difficulty)

    if not selected_questions:
         logging.warning(f"GET /questions/ failed: No questions found for criteria (difficulty: {difficulty}, count: {num_questions}).")
         raise HTTPException(status_code=404, detail=f"No questions found for the specified criteria (difficulty: {difficulty}, count: {num_questions}). Consider generating more questions and saving them to {JSON_FILE_PATH}.")

    logging.info(f"Successfully retrieved {len(selected_questions)} questions for /questions/.")
    return selected_questions

@app.post("/answer/")
def check_user_answer(answer: UserAnswer):
    """Check the user's answer against the correct answer."""
    logging.info(f"POST /answer/ request received for question_index={answer.question_index}, user_option={answer.user_option}")

    if answer.question_index < 0 or answer.question_index >= len(all_quiz_questions):
        logging.warning(f"POST /answer/ failed: Invalid question index received: {answer.question_index} (out of bounds).")
        raise HTTPException(status_code=400, detail="Invalid question index.")

    correct_question_data = all_quiz_questions[answer.question_index]
    correct_answer = correct_question_data.get("answer", "").strip().upper()

    if not correct_answer:
         logging.error(f"POST /answer/ failed: Could not retrieve correct answer for question index {answer.question_index}.")
         raise HTTPException(status_code=500, detail=f"Could not retrieve correct answer for question index {answer.question_index}.")


    is_correct = answer.user_option == correct_answer
    logging.info(f"POST /answer/ result for index {answer.question_index}: user_option={answer.user_option}, correct_answer={correct_answer}, is_correct={is_correct}")

    return {"question_index": answer.question_index, "user_answer": answer.user_option, "is_correct": is_correct, "correct_answer": correct_answer}

In [ ]:
# To run the FastAPI application within the notebook, you can use uvicorn.
# The --reload flag is useful during development to automatically restart the server
# when you make changes to the code.
# Note: This will block the cell execution.

import nest_asyncio
import uvicorn
import sys

# Apply nest_asyncio to allow the server to run within the notebook's event loop
nest_asyncio.apply()

print("Starting Uvicorn server...")

# Run the FastAPI application directly
# Assuming your FastAPI app is in a file named 'main.py' and the app instance is named 'app'
try:
    uvicorn.run("main:app", host="0.0.0.0", port=8000, reload=True)
    print("Uvicorn server stopped.")

except Exception as e:
    print(f"An error occurred while running uvicorn: {e}", file=sys.stderr)